In [55]:
"""
For 2 DIMMs results
"""
import pandas as pd
import numpy as np

def get_dual_results(path, nparts):
    buffer_distribution = []
    thread_epoch = []
    # Read the data from the file
    with open(path, "r") as f:
        for line in f.readlines():
            if (line.startswith("TIME|")):
                thread_epoch.append(line.strip("TIME|").strip("\n").split(","))
            else:
                buffer_distribution.append(line.strip("\n").split(","))

    if (buffer_distribution):
        buffer_distribution = np.array(buffer_distribution)
        buffer_distribution = pd.DataFrame(buffer_distribution, columns=['cceh0', 'cceh1', 'total'], dtype=np.float)

    if (thread_epoch):
        thread_epoch = np.array(thread_epoch)
        thread_epoch = pd.DataFrame(thread_epoch, columns=['thread', 'last_done', 'cur_done', 'seg_time', 'time'], dtype=np.float)
    
    # Find the MAX thread 
    max_thread = int(thread_epoch["thread"].max()) + 1
    thread_step = int(max_thread / nparts)

    """
    Calculate throughput : (epoch_ops/1/1024/1024)
    """
    max_execute_time = 0
    min_execute_time = 9999
    CCEH_max_execute_time = dict()
    for i in range(nparts): # Loop N CCEH
        start, end = i * thread_step, thread_step * (i + 1)
        local_execute_time = 0

        for j in range(start, end):
            execute_time = thread_epoch[thread_epoch["thread"]==j].shape[0]
            if (execute_time > local_execute_time):
                local_execute_time = execute_time
        CCEH_max_execute_time[i] = local_execute_time
        print(local_execute_time)
        
        if (local_execute_time < min_execute_time):
            min_execute_time = local_execute_time
        print(local_execute_time , max_execute_time)
        if (local_execute_time > max_execute_time):
            max_execute_time = local_execute_time
        print("thread ", start, " to ", end-1, "CCEH ", i, " runtime = ", local_execute_time)
        
    print ("All CCEH min runtime :", min_execute_time, "All CCEH max runtime :", max_execute_time)

    CCEH_Mops = dict()
    CCEH_time = dict()
    CCEH_throughput = dict()
    for i in range(nparts): # Loop N CCEH
        start, end = i * thread_step, thread_step * (i + 1)
        CCEH_Mops[i] = np.zeros(min_execute_time)
        CCEH_time[i] = np.array([k for k in range(1, min_execute_time + 1)])
        for j in range(start, end):
            thread_info = thread_epoch[thread_epoch["thread"]==j]
            epoch = thread_info['cur_done'] - thread_info['last_done']
            padding = np.pad(epoch.values, (0, CCEH_max_execute_time[i] - epoch.shape[0]), 'constant', constant_values=(0,0))
            CCEH_Mops[i] += padding[:min_execute_time]/1024/1024
        CCEH_throughput[i] = CCEH_Mops[i]
        print("CCEH Throughput ", i, " -> ", sum(CCEH_throughput[i]), "/", min_execute_time, sum(CCEH_throughput[i])/min_execute_time)

    overall_Mops = 0
    # Calculate overall throughput
    for idx in range(nparts):
        overall_Mops += sum(CCEH_Mops[i])
    print ("Overall throughput = ", overall_Mops / min_execute_time)    
    return CCEH_time, CCEH_throughput

In [56]:
CCEH_time0, CCEH_throughput0 = get_dual_results("./Dec1/data/cceh_dynamic_g5.txt", 2)
CCEH_time1, CCEH_throughput1 = get_dual_results("./Dec1/data/cceh_fixed_g5.txt", 2)

13
13 0
thread  0  to  19 CCEH  0  runtime =  13
6
6 13
thread  20  to  39 CCEH  1  runtime =  6
All CCEH min runtime : 6 All CCEH max runtime : 13
CCEH Throughput  0  ->  33.97941589355469 / 6 5.663235982259114
CCEH Throughput  1  ->  110.33058166503906 / 6 18.38843027750651
Overall throughput =  36.77686055501302
15
15 0
thread  0  to  19 CCEH  0  runtime =  15
8
8 15
thread  20  to  39 CCEH  1  runtime =  8
All CCEH min runtime : 8 All CCEH max runtime : 15
CCEH Throughput  0  ->  41.09382629394531 / 8 5.136728286743164
CCEH Throughput  1  ->  109.61532592773438 / 8 13.701915740966797
Overall throughput =  27.403831481933594
